# H&M Fashion Recommendation Pipeline

Ce notebook implémente un pipeline de recommandation pour les données de mode H&M en utilisant LightFM. Il comprend le chargement des données, le prétraitement, le filtrage collaboratif, l'optimisation des hyperparamètres et un modèle hybride intégrant des caractéristiques d'articles. Le pipeline évalue les performances du modèle et fournit un objet système de recommandation final pour faire des prédictions.

# 0. CELLULE DE CONFIGURATION - MODIFIEZ UNIQUEMENT CETTE SECTION

In [ ]:

QUICK_TEST_MODE = True  # True pour test rapide, False pour run complet

# Chemin de base vers vos données (MODIFIEZ CETTE LIGNE)
USER_BASE_PATH = "/content/drive/MyDrive/PSL/00-RecommanderSystem/h2m-recsys"

# Nom du dossier contenant les fichiers CSV (modifiez si différent)
USER_DATA_FOLDER = "data"

# Créer un dossier unique pour chaque exécution ?
CREATE_TIMESTAMPED_OUTPUT = True

# ========================================
# PARAMÈTRES AVANCÉS (OPTIONNEL)
# ========================================

# Échantillonnage (mode rapide)
USER_QUICK_MAX_USERS = 1000
USER_QUICK_MAX_ARTICLES = 500
USER_QUICK_MIN_TRANSACTIONS = 10
USER_QUICK_MIN_PURCHASES = 50
USER_QUICK_EPOCHS_COLLABORATIVE = 2
USER_QUICK_EPOCHS_HYBRID = 5

# Échantillonnage (mode complet) - None = toutes les données
USER_FULL_MAX_USERS = None
USER_FULL_MAX_ARTICLES = None
USER_FULL_MIN_TRANSACTIONS = 5
USER_FULL_MIN_PURCHASES = 10
USER_FULL_EPOCHS_COLLABORATIVE = 5
USER_FULL_EPOCHS_HYBRID = 50

# Features articles à utiliser (ajoutez/supprimez selon vos données)
USER_ARTICLE_FEATURES = ['product_type_name', 'product_group_name', 'colour_group_name']

# Loss functions à tester
USER_LOSS_FUNCTIONS = ['warp', 'bpr']  # Ajoutez 'logistic'

# Grille hyperparamètres (mode rapide)
USER_QUICK_PARAM_GRID = {
    'no_components': [64, 128],
    'learning_rate': [0.05, 0.1],
    'item_alpha': [1e-5, 1e-4],
    'user_alpha': [1e-5, 1e-4]
}

# Grille hyperparamètres (mode complet)
USER_FULL_PARAM_GRID = {
    'no_components': [64, 128, 256],
    'learning_rate': [0.01, 0.05, 0.1],
    'item_alpha': [0.0, 1e-6, 1e-5, 1e-4],
    'user_alpha': [0.0, 1e-6, 1e-5, 1e-4]
}

# ========================================
# 🔧 CONFIGURATION AUTOMATIQUE (NE PAS MODIFIER)
# ========================================

# Application automatique des paramètres selon le mode
if QUICK_TEST_MODE:
    print("🚀 MODE TEST RAPIDE ACTIVÉ")
    print("📝 Échantillons réduits et moins d'epochs pour validation rapide")
    MAX_USERS = USER_QUICK_MAX_USERS
    MAX_ARTICLES = USER_QUICK_MAX_ARTICLES
    EPOCHS_COLLABORATIVE = USER_QUICK_EPOCHS_COLLABORATIVE
    EPOCHS_HYBRID = USER_QUICK_EPOCHS_HYBRID
    MIN_TRANSACTIONS = USER_QUICK_MIN_TRANSACTIONS
    MIN_PURCHASES = USER_QUICK_MIN_PURCHASES
    PARAM_GRID = USER_QUICK_PARAM_GRID
else:
    print("🐌 MODE COMPLET ACTIVÉ")
    MAX_USERS = USER_FULL_MAX_USERS
    MAX_ARTICLES = USER_FULL_MAX_ARTICLES
    EPOCHS_COLLABORATIVE = USER_FULL_EPOCHS_COLLABORATIVE
    EPOCHS_HYBRID = USER_FULL_EPOCHS_HYBRID
    MIN_TRANSACTIONS = USER_FULL_MIN_TRANSACTIONS
    MIN_PURCHASES = USER_FULL_MIN_PURCHASES
    PARAM_GRID = USER_FULL_PARAM_GRID

# Configuration des chemins
BASE_PATH = USER_BASE_PATH
DATA_PATH = f"{BASE_PATH}/{USER_DATA_FOLDER}"
ARTICLE_FEATURES = USER_ARTICLE_FEATURES
LOSS_FUNCTIONS = USER_LOSS_FUNCTIONS

print("🛍️ PIPELINE COMPLET DE RECOMMANDATION H&M FASHION")
print("=" * 60)
print(f"📂 Chemin données: {DATA_PATH}")
print(f"👥 Max utilisateurs: {MAX_USERS}")
print(f"🎽 Max articles: {MAX_ARTICLES}")
print(f"🏷️ Features: {len(ARTICLE_FEATURES)}")

🚀 MODE TEST RAPIDE ACTIVÉ
📝 Échantillons réduits et moins d'epochs pour validation rapide
🛍️ PIPELINE COMPLET DE RECOMMANDATION H&M FASHION
📂 Chemin données: /content/drive/MyDrive/PSL/00-RecommanderSystem/h2m-recsys/data
👥 Max utilisateurs: 1000
🎽 Max articles: 500
🏷️ Features: 3


# 1. CONFIGURATION ET SETUP

In [ ]:
print("\n📗 1. CONFIGURATION ET SETUP")
print("-" * 30)

print("🔗 Connexion à Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive monté avec succès")

import os
from pathlib import Path
from datetime import datetime

print("\n🗂 Configuration des chemins de travail...")

# Création d'un répertoire unique pour chaque exécution (MODIFIÉ)
if CREATE_TIMESTAMPED_OUTPUT:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_id = f"run_{timestamp}"
    OUTPUTS_PATH = f"{BASE_PATH}/outputs/{run_id}"
else:
    OUTPUTS_PATH = f"{BASE_PATH}/outputs"

# Création du répertoire de sortie
os.makedirs(OUTPUTS_PATH, exist_ok=True)

print(f"📂 Chemin de base : {BASE_PATH}")
print(f"📂 Données : {DATA_PATH}")
print(f"📂 Sorties : {OUTPUTS_PATH}")
if CREATE_TIMESTAMPED_OUTPUT:
    print(f"🆔 ID d'exécution : {run_id}")

# Création d'un fichier de log pour cette exécution
log_file = f"{OUTPUTS_PATH}/execution_log.txt"
with open(log_file, 'w') as f:
    f.write(f"Exécution du pipeline H&M Fashion Recommendation\n")
    f.write(f"Date/Heure: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Mode: {'TEST RAPIDE' if QUICK_TEST_MODE else 'COMPLET'}\n")
    f.write(f"Répertoire: {OUTPUTS_PATH}\n")
    f.write(f"Paramètres utilisateur appliqués:\n")
    f.write(f"  - BASE_PATH: {BASE_PATH}\n")
    f.write(f"  - MAX_USERS: {MAX_USERS}\n")
    f.write(f"  - MAX_ARTICLES: {MAX_ARTICLES}\n")
    f.write(f"  - EPOCHS_COLLABORATIVE: {EPOCHS_COLLABORATIVE}\n")
    f.write(f"  - EPOCHS_HYBRID: {EPOCHS_HYBRID}\n")
    f.write("="*50 + "\n\n")

print(f"📝 Log d'exécution créé : {log_file}")

print("\n📦 Installation des bibliothèques...")
!pip install git+https://github.com/daviddavo/lightfm -q
!pip install -q tqdm

# Imports
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Configuration terminée")

# Fonction utilitaire pour logguer les étapes importantes
def log_step(step_name, details=""):
    """Log une étape dans le fichier de log"""
    timestamp = datetime.now().strftime('%H:%M:%S')
    with open(log_file, 'a') as f:
        f.write(f"[{timestamp}] {step_name}\n")
        if details:
            f.write(f"  {details}\n")
        f.write("\n")
    print(f"📝 Étape loggée: {step_name}")

log_step("Configuration terminée", f"Répertoire de sortie: {OUTPUTS_PATH}")


📗 1. CONFIGURATION ET SETUP
------------------------------
🔗 Connexion à Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive monté avec succès

🗂 Configuration des chemins de travail...
📂 Chemin de base : /content/drive/MyDrive/PSL/00-RecommanderSystem/h2m-recsys
📂 Données : /content/drive/MyDrive/PSL/00-RecommanderSystem/h2m-recsys/data
📂 Sorties : /content/drive/MyDrive/PSL/00-RecommanderSystem/h2m-recsys/outputs/run_20250928_113619
🆔 ID d'exécution : run_20250928_113619
📝 Log d'exécution créé : /content/drive/MyDrive/PSL/00-RecommanderSystem/h2m-recsys/outputs/run_20250928_113619/execution_log.txt

📦 Installation des bibliothèques...
  Preparing metadata (setup.py) ... done
✅ Configuration terminée
📝 Étape loggée: Configuration terminée


# 2. CHARGEMENT DES DONNÉES

In [ ]:
print("\n📊 2. CHARGEMENT DES DONNÉES")
print("-" * 30)

files_to_check = [
    f"{DATA_PATH}/transactions_train.csv",
    f"{DATA_PATH}/customers.csv",
    f"{DATA_PATH}/articles.csv"
]

for file_path in files_to_check:
    if os.path.exists(file_path):
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        print(f"✅ {os.path.basename(file_path)}: {size_mb:.1f} MB")
    else:
        print(f"❌ Fichier manquant: {os.path.basename(file_path)}")

print("\n⏳ Chargement en cours...")

try:
    transactions = pd.read_csv(f"{DATA_PATH}/transactions_train.csv")
    customers = pd.read_csv(f"{DATA_PATH}/customers.csv")
    articles = pd.read_csv(f"{DATA_PATH}/articles.csv")

    print(f"📈 Transactions: {len(transactions):,} lignes")
    print(f"👥 Clients: {len(customers):,} lignes")
    print(f"👕 Articles: {len(articles):,} lignes")

except FileNotFoundError as e:
    print(f"❌ Erreur de chargement: {e}")
    print(f"🔍 Vérifiez que le chemin est correct: {DATA_PATH}")
    raise


📊 2. CHARGEMENT DES DONNÉES
------------------------------
✅ transactions_train.csv: 3326.4 MB
✅ customers.csv: 197.5 MB
✅ articles.csv: 34.5 MB

⏳ Chargement en cours...
📈 Transactions: 31,788,324 lignes
👥 Clients: 1,371,980 lignes
👕 Articles: 105,542 lignes


# 3. EXPLORATION RAPIDE DES DONNÉES

In [ ]:
# =============================================================================
# 3. EXPLORATION RAPIDE DES DONNÉES
# =============================================================================
print("\n🔍 3. EXPLORATION RAPIDE DES DONNÉES")
print("-" * 30)

# Informations générales
print(f"📊 Période des transactions: {transactions['t_dat'].min()} à {transactions['t_dat'].max()}")
print(f"👥 Clients uniques: {transactions['customer_id'].nunique():,}")
print(f"👕 Articles uniques: {transactions['article_id'].nunique():,}")
print(f"💰 Prix médian: {transactions['price'].median():.2f}")

# Conversion de dates
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])

# =============================================================================
# 4. ÉCHANTILLONNAGE STRATÉGIQUE OPTIMISÉ
# =============================================================================
print("\n🎯 4. ÉCHANTILLONNAGE STRATÉGIQUE")
print("-" * 30)

# Analyse de l'activité des utilisateurs
user_activity = transactions.groupby('customer_id').size().sort_values(ascending=False)
print(f"👥 Clients uniques: {len(user_activity):,}")
print(f"📊 Transactions par client - Médiane: {user_activity.median():.0f}")

# Clients actifs (configuration optimisée)
active_users = user_activity[user_activity >= MIN_TRANSACTIONS].index
if QUICK_TEST_MODE and len(active_users) > MAX_USERS:
    active_users = active_users[:MAX_USERS]

# Articles populaires
article_popularity = transactions.groupby('article_id').size().sort_values(ascending=False)
popular_articles = article_popularity[article_popularity >= MIN_PURCHASES].index
if QUICK_TEST_MODE and len(popular_articles) > MAX_ARTICLES:
    popular_articles = popular_articles[:MAX_ARTICLES]

print(f"🔥 Clients actifs sélectionnés: {len(active_users):,}")
print(f"📈 Articles populaires sélectionnés: {len(popular_articles):,}")

# Création de l'échantillon
sample_transactions = transactions[
    (transactions['customer_id'].isin(active_users)) &
    (transactions['article_id'].isin(popular_articles))
].copy()

print(f"📊 Échantillon créé: {len(sample_transactions):,} transactions")
print(f"👥 Clients dans l'échantillon: {sample_transactions['customer_id'].nunique():,}")
print(f"👕 Articles dans l'échantillon: {sample_transactions['article_id'].nunique():,}")

# Sauvegarde
os.makedirs(OUTPUTS_PATH, exist_ok=True)
sample_data = {
    'transactions': sample_transactions,
    'customers': customers[customers['customer_id'].isin(sample_transactions['customer_id'])],
    'articles': articles[articles['article_id'].isin(sample_transactions['article_id'])]
}

sample_path = f"{OUTPUTS_PATH}/sample_data.pkl"
with open(sample_path, 'wb') as f:
    pickle.dump(sample_data, f)

print(f"💾 Échantillon sauvegardé: {sample_path}")
log_step("Échantillonnage terminé", f"Transactions: {len(sample_transactions):,}, Users: {len(active_users):,}, Articles: {len(popular_articles):,}")



🔍 3. EXPLORATION RAPIDE DES DONNÉES
------------------------------
📊 Période des transactions: 2018-09-20 à 2020-09-22
👥 Clients uniques: 1,362,281
👕 Articles uniques: 104,547
💰 Prix médian: 0.03

🎯 4. ÉCHANTILLONNAGE STRATÉGIQUE
------------------------------
👥 Clients uniques: 1,362,281
📊 Transactions par client - Médiane: 9
🔥 Clients actifs sélectionnés: 1,000
📈 Articles populaires sélectionnés: 500
📊 Échantillon créé: 41,850 transactions
👥 Clients dans l'échantillon: 998
👕 Articles dans l'échantillon: 500
💾 Échantillon sauvegardé: /content/drive/MyDrive/PSL/00-RecommanderSystem/h2m-recsys/outputs/run_20250928_113619/sample_data.pkl
📝 Étape loggée: Échantillonnage terminé


# 5. PRÉPARATION DES DONNÉES


In [ ]:
print("\n🔧 5. PRÉPARATION DES DONNÉES")
print("-" * 30)

# Récupération des données échantillonnées
transactions_sample = sample_data['transactions']
customers_sample = sample_data['customers']
articles_sample = sample_data['articles']

# Encodage des IDs
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

transactions_sample['user_id'] = user_encoder.fit_transform(transactions_sample['customer_id'])
transactions_sample['item_id'] = item_encoder.fit_transform(transactions_sample['article_id'])

num_users = len(user_encoder.classes_)
num_items = len(item_encoder.classes_)

print(f"👥 Utilisateurs encodés: {num_users:,}")
print(f"👕 Articles encodés: {num_items:,}")

# Split temporel train/test avec séparation stricte des interactions
split_date = transactions_sample['t_dat'].quantile(0.8)
train_transactions = transactions_sample[transactions_sample['t_dat'] <= split_date]
test_transactions = transactions_sample[transactions_sample['t_dat'] > split_date]

print(f"📅 Date de split: {split_date}")
print(f"🚂 Train: {len(train_transactions):,} transactions")
print(f"🧪 Test: {len(test_transactions):,} transactions")

# Création des matrices train/test
train_matrix = csr_matrix(
    (np.ones(len(train_transactions)),
     (train_transactions['user_id'], train_transactions['item_id'])),
    shape=(num_users, num_items)
)

# Pour le test, on ne garde que les nouvelles interactions (pas dans train)
# Créer un set des interactions d'entraînement
train_interactions_set = set(zip(train_transactions['user_id'], train_transactions['item_id']))

# Filtrer les interactions de test pour exclure celles déjà dans train
test_filtered = []
for _, row in test_transactions.iterrows():
    user_item_pair = (row['user_id'], row['item_id'])
    if user_item_pair not in train_interactions_set:
        test_filtered.append(row)

if test_filtered:
    test_df = pd.DataFrame(test_filtered)
    test_matrix = csr_matrix(
        (np.ones(len(test_df)),
         (test_df['user_id'], test_df['item_id'])),
        shape=(num_users, num_items)
    )
    print(f"🔍 Interactions test filtrées: {len(test_df):,} (nouvelles interactions uniquement)")
else:
    # Si pas d'interactions nouvelles, créer un split utilisateur différent
    print("⚠️  Pas de nouvelles interactions dans le test temporel, utilisation d'un split utilisateur...")

    # Split par utilisateur (80/20)
    unique_users = transactions_sample['user_id'].unique()
    np.random.seed(42)
    np.random.shuffle(unique_users)

    n_train_users = int(0.8 * len(unique_users))
    train_users = unique_users[:n_train_users]
    test_users = unique_users[n_train_users:]

    train_transactions = transactions_sample[transactions_sample['user_id'].isin(train_users)]
    test_transactions = transactions_sample[transactions_sample['user_id'].isin(test_users)]

    train_matrix = csr_matrix(
        (np.ones(len(train_transactions)),
         (train_transactions['user_id'], train_transactions['item_id'])),
        shape=(num_users, num_items)
    )

    test_matrix = csr_matrix(
        (np.ones(len(test_transactions)),
         (test_transactions['user_id'], test_transactions['item_id'])),
        shape=(num_users, num_items)
    )

    print(f"🚂 Train (utilisateurs): {len(train_transactions):,} transactions, {len(train_users):,} utilisateurs")
    print(f"🧪 Test (utilisateurs): {len(test_transactions):,} transactions, {len(test_users):,} utilisateurs")

print(f"📊 Matrice train: {train_matrix.shape} - Interactions: {train_matrix.nnz:,}")
print(f"📊 Matrice test: {test_matrix.shape} - Interactions: {test_matrix.nnz:,}")
print(f"📊 Densité train: {train_matrix.nnz / (train_matrix.shape[0] * train_matrix.shape[1]):.6f}")
print(f"📊 Densité test: {test_matrix.nnz / (test_matrix.shape[0] * test_matrix.shape[1]):.6f}")

# Vérification qu'il n'y a pas d'interactions communes
common_interactions = train_matrix.multiply(test_matrix).nnz
print(f"🔍 Interactions communes (doit être 0): {common_interactions}")

if common_interactions > 0:
    print("⚠️  ATTENTION: Il y a encore des interactions communes!")
    print("🔧 Application d'un nettoyage supplémentaire...")

    # Méthode alternative: masquer les interactions communes dans test
    test_matrix_clean = test_matrix.copy()
    test_matrix_clean[train_matrix.nonzero()] = 0
    test_matrix_clean.eliminate_zeros()
    test_matrix = test_matrix_clean

    print(f"✅ Test nettoyé: {test_matrix.nnz:,} interactions")
    print(f"🔍 Interactions communes après nettoyage: {train_matrix.multiply(test_matrix).nnz}")

assert train_matrix.multiply(test_matrix).nnz == 0, "Erreur: interactions communes détectées!"
print("✅ Validation: Aucune interaction commune entre train et test")

# Sauvegarde
prepared_data = {
    'train_matrix': train_matrix,
    'test_matrix': test_matrix,
    'user_encoder': user_encoder,
    'item_encoder': item_encoder,
    'transactions_sample': transactions_sample,
    'train_transactions': train_transactions,
    'test_transactions': test_transactions
}

prepared_path = f"{OUTPUTS_PATH}/prepared_data.pkl"
with open(prepared_path, 'wb') as f:
    pickle.dump(prepared_data, f)

print(f"💾 Données préparées sauvegardées")
log_step("Préparation données terminée", f"Train: {train_matrix.nnz:,} interactions, Test: {test_matrix.nnz:,} interactions")


🔧 5. PRÉPARATION DES DONNÉES
------------------------------
👥 Utilisateurs encodés: 998
👕 Articles encodés: 500
📅 Date de split: 2020-04-04 00:00:00
🚂 Train: 33,627 transactions
🧪 Test: 8,223 transactions
🔍 Interactions test filtrées: 6,781 (nouvelles interactions uniquement)
📊 Matrice train: (998, 500) - Interactions: 19,996
📊 Matrice test: (998, 500) - Interactions: 4,588
📊 Densité train: 0.040072
📊 Densité test: 0.009194
🔍 Interactions communes (doit être 0): 0
✅ Validation: Aucune interaction commune entre train et test
💾 Données préparées sauvegardées
📝 Étape loggée: Préparation données terminée


# 6. MODÈLE COLLABORATIF


In [ ]:
print("\n🤝 6. MODÈLE COLLABORATIF")
print("-" * 30)

# Test de différentes loss functions (MODIFIÉ pour utiliser LOSS_FUNCTIONS)
loss_functions = LOSS_FUNCTIONS
models = {}
results = {}

for loss in loss_functions:
    print(f"\n🔄 Entraînement avec {loss.upper()}...")

    model = LightFM(loss=loss, random_state=42)
    model.fit(train_matrix, epochs=EPOCHS_COLLABORATIVE, num_threads=2, verbose=True)

    # Évaluation
    train_auc = auc_score(model, train_matrix).mean()
    test_auc = auc_score(model, test_matrix, train_interactions=train_matrix).mean()
    test_precision = precision_at_k(model, test_matrix, train_interactions=train_matrix, k=10).mean()

    results[loss] = {
        'train_auc': train_auc,
        'test_auc': test_auc,
        'test_precision': test_precision
    }
    models[loss] = model

    print(f"✅ {loss.upper()} - AUC Test: {test_auc:.4f}, Precision@10: {test_precision:.4f}")

# Sélection du meilleur modèle
results_df = pd.DataFrame(results).T
print(f"\n📊 COMPARAISON DES MODÈLES:")
print(results_df)

best_loss = results_df['test_auc'].idxmax()
best_model = models[best_loss]
best_score = results_df.loc[best_loss, 'test_auc']

print(f"\n🏆 Meilleur modèle: {best_loss.upper()} (AUC: {best_score:.4f})")
log_step("Modèle collaboratif terminé", f"Meilleur: {best_loss.upper()}, AUC: {best_score:.4f}")


🤝 6. MODÈLE COLLABORATIF
------------------------------

🔄 Entraînement avec WARP...


Epoch: 100%|██████████| 2/2 [00:00<00:00,  4.56it/s]


✅ WARP - AUC Test: 0.4585, Precision@10: 0.0139

🔄 Entraînement avec BPR...


Epoch: 100%|██████████| 2/2 [00:00<00:00,  4.16it/s]


✅ BPR - AUC Test: 0.4973, Precision@10: 0.0096

📊 COMPARAISON DES MODÈLES:
      train_auc  test_auc  test_precision
warp   0.656556  0.458495        0.013901
bpr    0.541479  0.497327        0.009641

🏆 Meilleur modèle: BPR (AUC: 0.4973)
📝 Étape loggée: Modèle collaboratif terminé


# 7. OPTIMISATION DES HYPERPARAMÈTRES (RAPIDE)


In [ ]:
print("\n🔧 7. OPTIMISATION DES HYPERPARAMÈTRES")
print("-" * 30)

# Grille réduite pour test rapide (MODIFIÉ pour utiliser PARAM_GRID)
param_grid = PARAM_GRID

print(f"🔍 Test de {np.prod([len(v) for v in param_grid.values()])} combinaisons...")

best_optimized_score = 0
best_params = {}
optimization_count = 0

for no_components in param_grid['no_components']:
    for learning_rate in param_grid['learning_rate']:
        for item_alpha in param_grid['item_alpha']:
            for user_alpha in param_grid['user_alpha']:
                optimization_count += 1

                params = {
                    'no_components': no_components,
                    'learning_rate': learning_rate,
                    'item_alpha': item_alpha,
                    'user_alpha': user_alpha
                }

                try:
                    model = LightFM(
                        loss=best_loss,
                        no_components=no_components,
                        learning_rate=learning_rate,
                        item_alpha=item_alpha,
                        user_alpha=user_alpha,
                        random_state=42
                    )

                    model.fit(train_matrix, epochs=EPOCHS_COLLABORATIVE, num_threads=2)
                    test_auc = auc_score(model, test_matrix, train_interactions=train_matrix).mean()

                    if test_auc > best_optimized_score:
                        best_optimized_score = test_auc
                        best_params = params
                        best_optimized_model = model

                    print(f"✓ {optimization_count}/{np.prod([len(v) for v in param_grid.values()])} - Components: {no_components}, LR: {learning_rate}, AUC: {test_auc:.4f}")

                except Exception as e:
                    print(f"✗ Erreur avec {params}: {e}")

print(f"\n🏆 MEILLEURS HYPERPARAMÈTRES:")
for param, value in best_params.items():
    print(f"  {param}: {value}")
print(f"🎯 Meilleur score AUC: {best_optimized_score:.4f}")
log_step("Optimisation hyperparamètres terminée", f"Meilleur AUC: {best_optimized_score:.4f}, Params: {best_params}")



🔧 7. OPTIMISATION DES HYPERPARAMÈTRES
------------------------------
🔍 Test de 16 combinaisons...
✓ 1/16 - Components: 64, LR: 0.05, AUC: 0.5207
✓ 2/16 - Components: 64, LR: 0.05, AUC: 0.5226
✓ 3/16 - Components: 64, LR: 0.05, AUC: 0.5231
✓ 4/16 - Components: 64, LR: 0.05, AUC: 0.5229
✓ 5/16 - Components: 64, LR: 0.1, AUC: 0.5278
✓ 6/16 - Components: 64, LR: 0.1, AUC: 0.5275
✓ 7/16 - Components: 64, LR: 0.1, AUC: 0.5310
✓ 8/16 - Components: 64, LR: 0.1, AUC: 0.5314
✓ 9/16 - Components: 128, LR: 0.05, AUC: 0.5201
✓ 10/16 - Components: 128, LR: 0.05, AUC: 0.5204
✓ 11/16 - Components: 128, LR: 0.05, AUC: 0.5204
✓ 12/16 - Components: 128, LR: 0.05, AUC: 0.5205
✓ 13/16 - Components: 128, LR: 0.1, AUC: 0.5186
✓ 14/16 - Components: 128, LR: 0.1, AUC: 0.5186
✓ 15/16 - Components: 128, LR: 0.1, AUC: 0.5188
✓ 16/16 - Components: 128, LR: 0.1, AUC: 0.5186

🏆 MEILLEURS HYPERPARAMÈTRES:
  no_components: 64
  learning_rate: 0.1
  item_alpha: 0.0001
  user_alpha: 0.0001
🎯 Meilleur score AUC: 0.5314


# 8. ÉVALUATION COMPLÈTE


In [ ]:
print("\n📊 8. ÉVALUATION COMPLÈTE")
print("-" * 30)

# Évaluation sur différentes valeurs de k
k_values = [5, 10, 20]
evaluation_results = {}

print("📈 MÉTRIQUES D'ÉVALUATION")
for k in k_values:
    precision_k = precision_at_k(best_optimized_model, test_matrix, train_interactions=train_matrix, k=k).mean()
    recall_k = recall_at_k(best_optimized_model, test_matrix, train_interactions=train_matrix, k=k).mean()

    evaluation_results[f'precision@{k}'] = precision_k
    evaluation_results[f'recall@{k}'] = recall_k

    print(f"📊 Precision@{k}: {precision_k:.4f} | Recall@{k}: {recall_k:.4f}")

evaluation_results['auc'] = best_optimized_score
print(f"📊 AUC Score: {best_optimized_score:.4f}")
log_step("Évaluation modèle terminée", f"AUC: {best_optimized_score:.4f}, Precision@10: {evaluation_results['precision@10']:.4f}")



📊 8. ÉVALUATION COMPLÈTE
------------------------------
📈 MÉTRIQUES D'ÉVALUATION
📊 Precision@5: 0.0099 | Recall@5: 0.0085
📊 Precision@10: 0.0117 | Recall@10: 0.0190
📊 Precision@20: 0.0158 | Recall@20: 0.0613
📊 AUC Score: 0.5314
📝 Étape loggée: Évaluation modèle terminée


# 9. MODÈLE HYBRIDE


In [ ]:
print("\n🔀 9. MODÈLE HYBRIDE AVEC FEATURES")
print("-" * 30)

print("🏷️ Préparation des features d'articles...")

# Sélection des features catégorielles (MODIFIÉ pour utiliser ARTICLE_FEATURES)
feature_columns = [col for col in ARTICLE_FEATURES if col in articles_sample.columns]
if not feature_columns:
    print(f"❌ Aucune feature disponible parmi: {ARTICLE_FEATURES}")
    print(f"📋 Colonnes disponibles: {list(articles_sample.columns)}")
    # Utiliser des features par défaut si disponibles
    fallback_features = ['product_type_name', 'product_group_name', 'colour_group_name']
    feature_columns = [col for col in fallback_features if col in articles_sample.columns]
    if feature_columns:
        print(f"🔄 Utilisation des features par défaut: {feature_columns}")
    else:
        print("❌ Aucune feature utilisable trouvée - utilisation du modèle collaboratif")
        hybrid_model = best_optimized_model
        hybrid_auc = best_optimized_score
        hybrid_precision = evaluation_results['precision@10']
        feature_columns = []

if feature_columns:
    print(f"✅ Features utilisées: {feature_columns}")

    articles_features = articles_sample[['article_id'] + feature_columns].copy()

    # Nettoyage des valeurs manquantes
    for col in feature_columns:
        articles_features[col] = articles_features[col].fillna('Unknown')

    # Création du dataset LightFM avec features
    dataset = Dataset()

    unique_users = transactions_sample['customer_id'].unique()
    unique_items = transactions_sample['article_id'].unique()

    # Préparation des features d'items
    item_features = []
    for _, row in articles_features.iterrows():
        features = [f"{col}:{row[col]}" for col in feature_columns]
        item_features.append((row['article_id'], features))

    dataset.fit(users=unique_users,
               items=unique_items,
               item_features=[f"{col}:{val}" for col in feature_columns
                             for val in articles_features[col].unique()])

    print(f"📊 Dataset créé avec {len(unique_users):,} utilisateurs et {len(unique_items):,} articles")

    # Construction des matrices avec features
    interactions_matrix, weights = dataset.build_interactions(
        [(row['customer_id'], row['article_id']) for _, row in transactions_sample.iterrows()]
    )

    item_features_matrix = dataset.build_item_features(item_features)

    print(f"📊 Matrice d'interactions: {interactions_matrix.shape}")
    print(f"📊 Matrice de features: {item_features_matrix.shape}")

    # Entraînement du modèle hybride
    print(f"\n🔄 Entraînement du modèle hybride ({EPOCHS_HYBRID} epochs)...")

    hybrid_model = LightFM(
        loss=best_loss,
        no_components=best_params.get('no_components', 128),
        learning_rate=best_params.get('learning_rate', 0.05),
        item_alpha=best_params.get('item_alpha', 1e-5),
        user_alpha=best_params.get('user_alpha', 1e-5),
        random_state=42
    )

    hybrid_model.fit(interactions_matrix,
                    item_features=item_features_matrix,
                    epochs=EPOCHS_HYBRID,
                    num_threads=2,
                    verbose=True)

    # Évaluation du modèle hybride
    print("\n📊 Évaluation du modèle hybride...")

    hybrid_auc = auc_score(hybrid_model, interactions_matrix, item_features=item_features_matrix).mean()
    hybrid_precision = precision_at_k(hybrid_model, interactions_matrix, item_features=item_features_matrix, k=10).mean()

    print(f"🎯 Modèle Hybride - AUC: {hybrid_auc:.4f}")
    print(f"🎯 Modèle Hybride - Precision@10: {hybrid_precision:.4f}")

    # Comparaison
    print(f"\n📊 COMPARAISON MODÈLES:")
    print(f"  Collaboratif - AUC: {best_optimized_score:.4f}")
    print(f"  Hybride - AUC: {hybrid_auc:.4f}")
    improvement = ((hybrid_auc - best_optimized_score) / best_optimized_score * 100)
    print(f"  Amélioration: {improvement:+.2f}%")
    log_step("Modèle hybride terminé", f"AUC Hybride: {hybrid_auc:.4f}, Amélioration: {improvement:+.2f}%")

else:
    # Pas de features disponibles
    hybrid_auc = best_optimized_score
    hybrid_precision = evaluation_results['precision@10']
    improvement = 0.0
    print("📊 Utilisation du modèle collaboratif optimisé comme modèle final")


🔀 9. MODÈLE HYBRIDE AVEC FEATURES
------------------------------
🏷️ Préparation des features d'articles...
✅ Features utilisées: ['product_type_name', 'product_group_name', 'colour_group_name']
📊 Dataset créé avec 998 utilisateurs et 500 articles
📊 Matrice d'interactions: (998, 500)
📊 Matrice de features: (500, 565)

🔄 Entraînement du modèle hybride (5 epochs)...


Epoch: 100%|██████████| 5/5 [00:01<00:00,  2.59it/s]



📊 Évaluation du modèle hybride...
🎯 Modèle Hybride - AUC: 0.7111
🎯 Modèle Hybride - Precision@10: 0.1763

📊 COMPARAISON MODÈLES:
  Collaboratif - AUC: 0.5314
  Hybride - AUC: 0.7111
  Amélioration: +33.80%
📝 Étape loggée: Modèle hybride terminé


# 10. PIPELINE FINAL ET TESTS


In [ ]:
print("\n🚀 10. PIPELINE FINAL ET TESTS")
print("-" * 30)

# Classe pour le système de recommandation
class HMRecommendationSystem:
    def __init__(self, model, dataset=None, item_features_matrix=None, user_encoder=None, item_encoder=None):
        self.model = model
        self.dataset = dataset
        self.item_features_matrix = item_features_matrix
        self.user_encoder = user_encoder
        self.item_encoder = item_encoder
        self.is_hybrid = dataset is not None and item_features_matrix is not None

    def get_user_recommendations(self, customer_id, n_recommendations=10):
        """Obtient des recommandations pour un utilisateur"""
        try:
            if self.is_hybrid:
                # Mode hybride - utilise le dataset LightFM
                user_mapping = self.dataset.mapping()[0]
                item_mapping = self.dataset.mapping()[2]

                if customer_id not in user_mapping:
                    print(f"Utilisateur {customer_id} non trouvé dans le dataset hybride")
                    return []

                user_id = user_mapping[customer_id]
                item_ids = list(range(len(item_mapping)))
                user_ids = [user_id] * len(item_ids)

                scores = self.model.predict(user_ids, item_ids, item_features=self.item_features_matrix)

                top_items = np.argsort(scores)[::-1][:n_recommendations]
                reverse_item_mapping = {v: k for k, v in item_mapping.items()}
                top_articles = [reverse_item_mapping[i] for i in top_items]
                top_scores = scores[top_items]

            else:
                # Mode collaboratif classique
                user_id = self.user_encoder.transform([customer_id])[0]

                # Créer des arrays pour tous les articles
                user_ids = np.array([user_id] * len(self.item_encoder.classes_))
                item_ids = np.arange(len(self.item_encoder.classes_))

                scores = self.model.predict(user_ids, item_ids)

                top_items = np.argsort(scores)[::-1][:n_recommendations]
                top_articles = self.item_encoder.inverse_transform(top_items)
                top_scores = scores[top_items]

            return list(zip(top_articles, top_scores))

        except (ValueError, KeyError) as e:
            print(f"Erreur pour utilisateur {customer_id}: {e}")
            return []

    def get_similar_items(self, article_id, n_similar=10):
        """Trouve des articles similaires basés sur les embeddings"""
        try:
            if self.is_hybrid:
                # Mode hybride
                item_mapping = self.dataset.mapping()[2]
                if article_id not in item_mapping:
                    print(f"Article {article_id} non trouvé dans le dataset hybride")
                    return []

                item_id = item_mapping[article_id]
            else:
                # Mode collaboratif
                item_id = self.item_encoder.transform([article_id])[0]

            # Obtenir les embeddings d'articles
            item_embeddings = self.model.item_embeddings

            # Calculer la similarité cosinus
            target_embedding = item_embeddings[item_id]
            similarities = np.dot(item_embeddings, target_embedding)

            # Articles les plus similaires (exclure l'article lui-même)
            similar_items = np.argsort(similarities)[::-1][1:n_similar+1]

            if self.is_hybrid:
                reverse_item_mapping = {v: k for k, v in item_mapping.items()}
                similar_articles = [reverse_item_mapping[i] for i in similar_items]
            else:
                similar_articles = self.item_encoder.inverse_transform(similar_items)

            similarity_scores = similarities[similar_items]

            return list(zip(similar_articles, similarity_scores))

        except (ValueError, KeyError) as e:
            print(f"Erreur pour article {article_id}: {e}")
            return []

# Initialisation du système
print("🔧 Initialisation du système de recommandation...")

if feature_columns:
    # Système hybride
    recommender = HMRecommendationSystem(
        model=hybrid_model,
        dataset=dataset,
        item_features_matrix=item_features_matrix,
        user_encoder=user_encoder,
        item_encoder=item_encoder
    )
    print("✅ Système de recommandation hybride initialisé")
else:
    # Système collaboratif
    recommender = HMRecommendationSystem(
        model=best_optimized_model,
        user_encoder=user_encoder,
        item_encoder=item_encoder
    )
    print("✅ Système de recommandation collaboratif initialisé")

# Tests du système
print("\n🧪 TESTS DU SYSTÈME DE RECOMMANDATION")

# Test avec quelques utilisateurs
test_users = transactions_sample['customer_id'].unique()[:3]

for customer_id in test_users:
    print(f"\n👤 Recommandations pour le client {customer_id}:")
    recommendations = recommender.get_user_recommendations(customer_id, 3)

    if recommendations:
        for i, (article_id, score) in enumerate(recommendations[:3], 1):
            article_info = articles_sample[articles_sample['article_id'] == article_id]
            if not article_info.empty and 'product_type_name' in article_info.columns:
                product_name = article_info.iloc[0].get('product_type_name', 'N/A')
                color = article_info.iloc[0].get('colour_group_name', 'N/A')
                print(f"  {i}. Article {article_id} - {product_name} ({color}) - Score: {score:.3f}")
            else:
                print(f"  {i}. Article {article_id} - Score: {score:.3f}")
    else:
        print("  Aucune recommandation disponible")

# Test similarité d'articles
print(f"\n🔍 TEST SIMILARITÉ D'ARTICLES")
test_article = transactions_sample['article_id'].iloc[0]
similar_items = recommender.get_similar_items(test_article, 3)

if similar_items:
    print(f"Articles similaires à {test_article}:")
    for i, (article_id, similarity) in enumerate(similar_items[:3], 1):
        print(f"  {i}. Article {article_id} - Similarité: {similarity:.3f}")
else:
    print("Aucun article similaire trouvé")

# Sauvegarde finale (MODIFIÉ pour inclure la configuration utilisateur)
final_results = {
    'user_configuration': {
        'BASE_PATH': BASE_PATH,
        'DATA_PATH': DATA_PATH,
        'QUICK_TEST_MODE': QUICK_TEST_MODE,
        'MAX_USERS': MAX_USERS,
        'MAX_ARTICLES': MAX_ARTICLES,
        'EPOCHS_COLLABORATIVE': EPOCHS_COLLABORATIVE,
        'EPOCHS_HYBRID': EPOCHS_HYBRID,
        'ARTICLE_FEATURES': ARTICLE_FEATURES,
        'LOSS_FUNCTIONS': LOSS_FUNCTIONS,
        'feature_columns_used': feature_columns
    },
    'recommender': recommender,
    'collaborative_auc': best_optimized_score,
    'hybrid_auc': hybrid_auc,
    'improvement': improvement,
    'best_params': best_params,
    'evaluation_results': evaluation_results,
    'model_type': 'hybrid' if feature_columns else 'collaborative'
}

final_path = f"{OUTPUTS_PATH}/final_recommendation_system.pkl"
with open(final_path, 'wb') as f:
    pickle.dump(final_results, f)

log_step("Pipeline terminé avec succès", f"Fichiers sauvegardés dans: {OUTPUTS_PATH}")

# =============================================================================
# RÉSUMÉ FINAL
# =============================================================================
print("\n" + "="*60)
print("🎉 PIPELINE TERMINÉ AVEC SUCCÈS !")
print("="*60)

print(f"\n📋 CONFIGURATION UTILISÉE:")
print(f"  🔧 Mode: {'TEST RAPIDE' if QUICK_TEST_MODE else 'COMPLET'}")
print(f"  📂 Chemin données: {DATA_PATH}")
print(f"  👥 Max utilisateurs: {MAX_USERS}")
print(f"  🎽 Max articles: {MAX_ARTICLES}")
print(f"  🏷️ Features utilisées: {len(feature_columns) if feature_columns else 0}")
print(f"  🤖 Type de modèle: {'Hybride' if feature_columns else 'Collaboratif'}")

print(f"\n📊 RÉSULTATS FINAUX:")
print(f"  👥 Utilisateurs traités: {num_users:,}")
print(f"  👕 Articles traités: {num_items:,}")
print(f"  🤝 Modèle collaboratif AUC: {best_optimized_score:.4f}")
print(f"  🔀 Modèle final AUC: {hybrid_auc:.4f}")
if feature_columns:
    print(f"  📈 Amélioration hybride: {improvement:+.2f}%")

print(f"\n💾 FICHIERS SAUVEGARDÉS DANS: {OUTPUTS_PATH}")
print(f"  📁 sample_data.pkl")
print(f"  📁 prepared_data.pkl")
print(f"  📁 final_recommendation_system.pkl")
print(f"  📁 execution_log.txt")

if CREATE_TIMESTAMPED_OUTPUT:
    print(f"\n🆔 ID EXÉCUTION: {run_id}")

print("✅ Système de recommandation opérationnel")
print("✅ Tests de validation effectués")

if QUICK_TEST_MODE:
    print("\n💡 Pour un run complet, changez QUICK_TEST_MODE = False dans la config")

print(f"\n🎯 RECOMMANDATIONS POUR LA PROCHAINE ÉTAPE:")
if not feature_columns:
    print("  📋 Vérifiez la disponibilité des features d'articles dans vos données")
    print("  🔧 Ajustez USER_ARTICLE_FEATURES selon vos colonnes disponibles")
if QUICK_TEST_MODE:
    print("  🚀 Testez en mode complet pour de meilleures performances")
print("  📊 Analysez les résultats dans le fichier de log")

print("\n🚀 Pipeline prêt pour la production !")

# Finalisation du log (MODIFIÉ pour inclure la configuration)
with open(log_file, 'a') as f:
    f.write("="*50 + "\n")
    f.write(f"EXÉCUTION TERMINÉE À: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"CONFIGURATION FINALE:\n")
    f.write(f"  - Mode: {'TEST RAPIDE' if QUICK_TEST_MODE else 'COMPLET'}\n")
    f.write(f"  - Chemin données: {DATA_PATH}\n")
    f.write(f"  - Features utilisées: {feature_columns}\n")
    f.write(f"RÉSULTATS FINAUX:\n")
    f.write(f"  - Utilisateurs: {num_users:,}\n")
    f.write(f"  - Articles: {num_items:,}\n")
    f.write(f"  - AUC Collaboratif: {best_optimized_score:.4f}\n")
    f.write(f"  - AUC Final: {hybrid_auc:.4f}\n")
    if feature_columns:
        f.write(f"  - Amélioration: {improvement:+.2f}%\n")
    f.write(f"  - Type modèle: {'Hybride' if feature_columns else 'Collaboratif'}\n")

print(f"\n📝 Log complet disponible: {log_file}")

# =============================================================================
# 📋 GUIDE RAPIDE POUR L'UTILISATEUR
# =============================================================================
print("\n" + "="*60)
print("📋 GUIDE RAPIDE POUR UTILISER CE PIPELINE")
print("="*60)

print(f"""
🔧 POUR PERSONNALISER LE PIPELINE:

1. CHANGEZ CES VARIABLES AU DÉBUT DU CODE:
   • USER_BASE_PATH = "/votre/chemin/vers/données"
   • USER_DATA_FOLDER = "nom_dossier_csv"
   • QUICK_TEST_MODE = True/False

2. AJUSTEZ LES PARAMÈTRES SELON VOS BESOINS:
   • USER_QUICK_MAX_USERS = nombre d'utilisateurs max
   • USER_ARTICLE_FEATURES = ['colonne1', 'colonne2', ...]
   • USER_LOSS_FUNCTIONS = ['warp', 'bpr', 'logistic']

3. LANCEZ LE CODE ET VÉRIFIEZ LES RÉSULTATS DANS:
   {OUTPUTS_PATH}

🚀 PRÊT À UTILISER - AUCUNE AUTRE MODIFICATION NÉCESSAIRE!
""")

print("="*60)


🚀 10. PIPELINE FINAL ET TESTS
------------------------------
🔧 Initialisation du système de recommandation...
✅ Système de recommandation hybride initialisé

🧪 TESTS DU SYSTÈME DE RECOMMANDATION

👤 Recommandations pour le client 01e464bf74b13a55df22de1528eff2b33749c0cd92953b62bd22dee2de17d1fd:
  1. Article 448509014 - Trousers (Blue) - Score: -0.553
  2. Article 714790003 - Trousers (Blue) - Score: -0.595
  3. Article 573085004 - Trousers (Blue) - Score: -0.597

👤 Recommandations pour le client 08ddd7be3e60252d2cba9dd55297a6ad0bdc1f4e244a16a8ebf61b73c56557c5:
  1. Article 706016001 - Trousers (Black) - Score: 0.071
  2. Article 562245046 - Trousers (Black) - Score: 0.070
  3. Article 399201005 - Trousers (Dark Blue) - Score: 0.062

👤 Recommandations pour le client 0e1ca3cd38b0fd1727ef7cc963d8f5ac1f37617593d57f1b497ffa74b0937c2c:
  1. Article 160442010 - Socks (White) - Score: -1.115
  2. Article 372860002 - Socks (White) - Score: -1.126
  3. Article 470789019 - Underwear bottom (Light